In [18]:
from pyspark.sql import SQLContext
sqlContext =SQLContext(sc)
df = sqlContext.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.printSchema()
df.registerTempTable("tweets")
data=sqlContext.sql("SELECT _id,created_at,text,entities.hashtags.text as hashtags from tweets limit 1000")


root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: null (nullable = true)
 |-- coordinates: null (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- ele

In [19]:
rdd1=data.rdd
rdd2=rdd1.map(lambda r:[r.text])
from textblob_fr import PatternTagger, PatternAnalyzer
from textblob import TextBlob
from textblob import Blobber
import nltk
tb =Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())


#text1=u"Valeurs Quelle est votre position Monsieur"
#blob1 = TextBlob(text1, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
#blob1.sentiment
#blob2 = tb(u"Marine a fait une campagne horrible")
#blob2.sentiment
#blob5=blob2.translate(to="en")
#blob5
#blob5.sentiment'''

In [20]:
#Création des listes de #hastags et de nuances nommées "liste_hashtags" et "liste_nuances"
rdd_nuances=sc.textFile("file:///media/sf_Fichiers_partages/Projet/allhash_nuances2.csv")
rdd_hashtags_nuances=rdd_nuances.map(lambda x:(x.replace(' #','#').replace(' @','@').split(';')))
rdd_hashtags=rdd_hashtags_nuances.map(lambda x:(x[0]))
liste_hashtags=rdd_hashtags.collect()
rdd_nuances=rdd_hashtags_nuances.map(lambda x:(x[1]))
liste_nuances=rdd_nuances.collect()

In [21]:
#Création du dictionnaire de #hastags nommée "dico_hash_nuances"
dico_hash_nuances={}
i=0
while i<len(liste_hashtags):
    dico_hash_nuances[liste_hashtags[i]]=liste_nuances[i]
    i+=1
dico_hash_nuances

{u'#AVENIRENCOMMUN': u'FI',
 u'#DLF': u'DLF',
 u'#EELV': u'ECO',
 u'#EM': u'REM',
 u'#FDG': u'COM',
 u'#FI': u'FI',
 u'#FN': u'FN',
 u'#FRANCEINSOUMISE': u'FI',
 u'#FREXIT': u'DIV',
 u'#INSOUMIS': u'FI',
 u'#INSOUMIS2017': u'FI',
 u'#LAREPUBLIQUEENMARCHE': u'REM',
 u'#LEGISLATIVESFI': u'FI',
 u'#LEGISLATIVESINSOUMISES': u'FI',
 u'#LESREPUBLICAINS ': u'LR',
 u'#LR': u'LR',
 u'#LRME': u'REM',
 u'#MAJORITEPOURLAFRANCE': u'REM',
 u'#MLP': u'FN',
 u'#MODEM': u'MDM',
 u'#NDA': u'DLF',
 u'#PARTISOCIALISTE': u'SOC',
 u'#PATRIOTE ': u'FN',
 u'#PCF': u'COM',
 u'#PS': u'SOC',
 u'#RESEAUFDG': u'COM',
 u'#UDI': u'UDI',
 u'#UPR': u'DIV',
 u'@DLF_OFFICIEL': u'DLF',
 u'@EELV': u'ECO',
 u'@ENMARCHEFR': u'REM',
 u'@FDG': u'COM',
 u'@FN': u'FN',
 u'@FN_OFFICIEL': u'FN',
 u'@FRANCEINSOUMISE': u'FI',
 u'@LR': u'LR',
 u'@MLP_OFFICIEL': u'FN',
 u'@UDI_OFF': u'UDI',
 u'ABBASSI': u'ECO',
 u'AESCHLIMANN': u'LR',
 u'ALBIGNAC': u'DIV',
 u'ALIX': u'SOC',
 u'ALLAERT': u'COM',
 u'AMRANI': u'FI',
 u'ANOU': u'FN',
 u'

In [22]:
def retourne_hashtag(texte,liste=liste_hashtags):
    texte2=texte.upper()
    listetext=texte2.replace(',',' ').replace('\'',' ').split(' ')
    liste_ind=[]
    liste_hash=[]
    for elt in listetext:
        if elt in liste:
            liste_ind.append(liste.index(elt))
    for i in liste_ind:
        liste_hash.append(liste[i])
    return liste_hash

In [23]:
def retourne_nuance(texte,dico=dico_hash_nuances):
    liste_nuances2=[]
    liste_hasht=retourne_hashtag(texte)
    for elt in liste_hasht:
        liste_nuances2.append(dico_hash_nuances[elt])
    return liste_nuances2

In [24]:
#Sélection
rdd3=rdd2.map(lambda x:(tb(x[0]).translate(to="en"),retourne_hashtag(x[0]),retourne_nuance(x[0])))
rdd3.take(10)

[(TextBlob("Predictive justice: "big data" hits the door of the courts and law firms https://t.co/ussrlylqEA via @franceinfo"),
  [],
  []),
 (TextBlob("RT @ GrosBill92: @PAAngade meets fellow citizens in Brussels. # FDE04 #Website #LREM # legislatives2017 https://t.co/mHACY1Js4H"),
  [],
  []),
 (TextBlob("@GG_RMC @MLP_officiel @etienneliebig The two Etienne ... the two ...!"),
  [u'@MLP_OFFICIEL'],
  [u'FN']),
 (TextBlob("RT @RMCinfo: @EmmanuelMacron 2nd debate #BourdinBDV at 9:30 am 📢: is the management of @ France2tv right to oust David Pujadas? Vote! ..."),
  [],
  []),
 (TextBlob("RT @ JSmi20: # Legislative DO NOT YOU LEAVE IMPRESSION BY "VOICE ATTRAP" MACRON #HULOT: IT CAN NOT PUT HIS POLI ..."),
  [],
  []),
 (TextBlob("RT @MaximCombes: The Ni of the right nor of the left seen by @rodho_g
  #PremierMinistre #Macron #gouuvernementphilippe https://t.co/QfuszmbPzg"),
  [],
  []),
 (TextBlob("Super video of the candidate @FranceInsummizes the French of Northern Europe on the conseq

In [31]:
#rdd4=rdd3.map(lambda x:(str(x[0]),(x[0].sentiment[0],x[0].sentiment[1]),set(x[2])))
rdd4=rdd3.map(lambda x:(((x[0].sentiment[0],x[0].sentiment[1]),x[1],set(x[2]))))
rdd4.take(10)

[((0.0, 0.1), [], set()),
 ((0.0, 0.0), [], set()),
 ((0.0, 0.0), [u'@MLP_OFFICIEL'], {u'FN'}),
 ((0.17857142857142855, 0.26785714285714285), [], set()),
 ((0.0, 0.0), [], set()),
 ((0.14285714285714285, 0.26785714285714285), [], set()),
 ((0.16666666666666666, 0.3333333333333333), [u'@FRANCEINSOUMISE'], {u'FI'}),
 ((0.10416666666666666, 0.19999999999999998), [], set()),
 ((0.0, 0.125), [], set()),
 ((0.0, 0.0), [], set())]

In [33]:
#Filtre sur les nuances, on n'en veut qu'une nuance par tweet
rdd5_res=rdd4.filter(lambda x:(len(x[2])==1))
rdd5_res.take(20)

[((0.0, 0.0), [u'@MLP_OFFICIEL'], {u'FN'}),
 ((0.16666666666666666, 0.3333333333333333), [u'@FRANCEINSOUMISE'], {u'FI'}),
 ((0.0, 0.0), [u'#FN'], {u'FN'}),
 ((0.018939393939393923, 0.6363636363636364), [u'@ENMARCHEFR'], {u'REM'}),
 ((0.6166666666666667, 0.8333333333333333), [u'COMOY'], {u'FI'}),
 ((0.0, 0.0), [u'#LR'], {u'LR'}),
 ((-0.4, 0.4), [u'#PS'], {u'SOC'}),
 ((0.2, 0.2), [u'COMOY'], {u'FI'}),
 ((0.0, 0.0), [u'#LAREPUBLIQUEENMARCHE'], {u'REM'}),
 ((-0.05000000000000001, 0.6666666666666666), [u'#PS'], {u'SOC'}),
 ((0.09285714285714285, 0.5178571428571428), [u'#FN'], {u'FN'}),
 ((0.0, 0.0), [u'#LR', u'#LR'], {u'LR'}),
 ((-0.8, 0.9), [u'#FI'], {u'FI'}),
 ((0.234375, 0.6499999999999999), [u'#FN'], {u'FN'}),
 ((0.0, 0.0), [u'#LR'], {u'LR'}),
 ((0.0, 0.0), [u'#LAREPUBLIQUEENMARCHE'], {u'REM'}),
 ((0.0, 0.0), [u'#LAREPUBLIQUEENMARCHE'], {u'REM'}),
 ((0.0, 0.0), [u'#FN'], {u'FN'}),
 ((0.0, 0.0), [u'#FN'], {u'FN'})]

In [34]:
#Renvoie une rdd avec la liste des mots-clés triés (en 1er les candidats, puis les partis. On renvoit le 1er elt)
rdd6_res=rdd5_res.map(lambda x:(x[0][0],list(reversed(sorted(x[1]))),str(x[2].pop())))\
.map(lambda x:(x[0],str(x[1][0]),x[2]))
                #(x[1][1][0] if (x[1][1][0]!='@' and x[1][0][0]!='#') else 3),x[2]))
rdd6_res.take(10)

[(0.0, '@MLP_OFFICIEL', 'FN'),
 (0.16666666666666666, '@FRANCEINSOUMISE', 'FI'),
 (0.0, '#FN', 'FN'),
 (0.018939393939393923, '@ENMARCHEFR', 'REM'),
 (0.6166666666666667, 'COMOY', 'FI'),
 (0.0, '#LR', 'LR'),
 (-0.4, '#PS', 'SOC'),
 (0.2, 'COMOY', 'FI'),
 (0.0, '#LAREPUBLIQUEENMARCHE', 'REM'),
 (-0.05000000000000001, '#PS', 'SOC')]

In [35]:
#FILTRE SUR LES CANDIDATS
rdd_cand=rdd6_res.filter(lambda x: x[1][0]!='@' and x[1][0]!='#')
rdd_cand.collect()

[(0.6166666666666667, 'COMOY', 'FI'), (0.2, 'COMOY', 'FI')]

In [36]:
#RDDPAIR avec key = Candidat
rdd_kv_cand=rdd_cand.keyBy(lambda x:x[1])
rdd_kv_cand_1=rdd_kv_cand.mapValues(lambda x:(x[0],1))
rdd_kv_cand_1.collect()


[('COMOY', (0.6166666666666667, 1)), ('COMOY', (0.2, 1))]

In [37]:
#RDD pair donnant le CANDIDAT , ( sa moyenne des sentiments , son nombre de tweets)
rdd_cand_res=rdd_kv_cand_1.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))\
.mapValues(lambda x: (x[0]/x[1],x[1]))
rdd_cand_res.collect()

[('COMOY', (0.4083333333333333, 2))]

In [38]:
#RDDPAIR avec key = Nuances
rdd_kv_nuances=rdd6_res.keyBy(lambda x:x[2])
rdd_kv_nuances_1=rdd_kv_nuances.mapValues(lambda x:(x[0],1))
rdd_kv_nuances_1.collect()

[('FN', (0.0, 1)),
 ('FI', (0.16666666666666666, 1)),
 ('FN', (0.0, 1)),
 ('REM', (0.018939393939393923, 1)),
 ('FI', (0.6166666666666667, 1)),
 ('LR', (0.0, 1)),
 ('SOC', (-0.4, 1)),
 ('FI', (0.2, 1)),
 ('REM', (0.0, 1)),
 ('SOC', (-0.05000000000000001, 1)),
 ('FN', (0.09285714285714285, 1)),
 ('LR', (0.0, 1)),
 ('FI', (-0.8, 1)),
 ('FN', (0.234375, 1)),
 ('LR', (0.0, 1)),
 ('REM', (0.0, 1)),
 ('REM', (0.0, 1)),
 ('FN', (0.0, 1)),
 ('FN', (0.0, 1))]

In [39]:
#RDD pair donnant la Nuance , ( sa moyenne des sentiments , son nombre de tweets)
rdd_parti_res=rdd_kv_nuances_1.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))\
.mapValues(lambda x: (x[0]/x[1],x[1]))
rdd_parti_res.collect()

[('FI', (0.04583333333333334, 4)),
 ('SOC', (-0.225, 2)),
 ('LR', (0.0, 3)),
 ('FN', (0.05453869047619048, 6)),
 ('REM', (0.004734848484848481, 4))]